In [1]:
import pandas as pd

from wsdata.consts import SPZZ, BUHANLIGAO
from wsdata.models import WinsunDatabase
from wsdata.utils import wan, Month

ws = WinsunDatabase()
m = Month()

def color_of_price(x):
    x = float(x)
    if x <= 2e4:
        return 'background-color: red'
    elif x <= 3e4:
        return 'background-color: green'
    elif x <= 4e4:
        return 'background-color: blue'
    else:
        return 'background-color: purple'

class ShiChang:
    gxj = ws.gxj('trend', 'month', 12, usage=SPZZ, plate=BUHANLIGAO)
    
    
    def __cumsum(self, by, group_by):
        df = ws.query(f'month_{by}').filter(usage=SPZZ, plate=BUHANLIGAO).group(group_by, '面积').df()
        df.面积 = df.面积.astype(float)
        return df
    
    def _stock(self, by):
        def cumsum(table): return self.__cumsum(table, by)
        sale = cumsum('sale') 
        sold = cumsum('sold')
        df = pd.merge(sale, sold, on=by)
        df['面积'] = df.面积_x - df.面积_y
        return df
    
    @property
    def trend_table(self):
        return self.gxj.df_adjusted

    @property
    def trend_argue(self):
        argue = self.gxj.shuoli(0).full_text
        
        stock = self._stock('presaleid').面积.sum()
        sold_speed = self.gxj.df_original.sold.mean()
        sold_cycle = stock / sold_speed
        
        argue += f'截止{m.month}月底全市库存{wan(stock):.0f}万㎡，去化周期{round(sold_cycle,0)}个月（按12个月计算）。'
        return argue
    
    @property
    def trend_average(self):
        df = self.gxj.df_original
        sale = df.sale.mean()
        sold = df.sold.mean()
        print(f'近一年月均供应：{wan(sale):.0f}万㎡', 
              f'近一年月均成交：{wan(sold):.0f}万㎡')
        
    @property
    def __stock_table_detail(self):
        def sale(period):
            df = ws.query('month_sold')\
                .filter(usage=SPZZ, plate=BUHANLIGAO, date_range=[m.before(period-1).date, m.date])\
                .group('面积段', '面积').df()
            df.rename(columns={'面积': f'面积_{period}'}, inplace=True)
            df.set_index('面积段', inplace=True)
            return df

        # 面积段标签
        cut_label = ws.query('month_sold')._cut_label('acreage', [90,120,144,180]).set_index('面积段')

        # 库存
        stock = self._stock('面积段').set_index('面积段')

        # 拼合 面积段、库存、3类去化量，按面积段分组
        df = cut_label.join([stock, sale(3), sale(12), sale(36)])
        
        return df
    
    @property
    def stock_table(self):
        df = self.__stock_table_detail
        df = df.pivot_table(index='acreage', aggfunc=sum)

        # 计算去化速度
        for i in [3,12,36]:
            df[f'速度_{i}'] = df.面积 / df[f'面积_{i}']

        # 调整
        df.面积 = df.面积.apply(wan)
        df = df[['面积','速度_3','速度_12','速度_36']].round(2)
        df.index = [f'{x}㎡' for x in df.index]
        
        return df
    
    @property
    def stock_argue(self):
        df = self.__stock_table_detail
        
        stock = df.面积.sum()
        cycle = [stock / df[f'面积_{i}'].sum() for i in (3,12,36)]
        
        return f'截止本月底库存：{wan(stock):.0f}万㎡'\
               f'   去化周期—按近3个月计算：{cycle[0]:.1f}个月'\
               f'   按近12个月计算：{cycle[1]:.1f}个月'\
               f'   按近36个月计算：{cycle[2]:.1f}个月'
    
    
    @property
    def cross_table(self):
        df = ws.query('month_sold')\
                .filter(usage=SPZZ, plate=BUHANLIGAO, 
                        date_range=[m.date, m.date])\
                .cross(values='件数', margins=True,
                       idx_cols=['tprice', 'acreage'],
                       idx_bins=[100,120,150,200,250,300,350,400,450,500], 
                       cols_bins=[90,120,144,180])
        df = df.fillna(0).astype(int)
        return df.style.background_gradient()
    
    @property
    def rank(self):
        df = ws.rank('month_sold', 1, ['popularizename','板块'], ['面积','件数','金额','均价']).head(10)
        df[['面积','金额']] = (df[['面积','金额']]/[1e4, 1e8]).round(2)
        df.均价 = df.均价.astype(int)
        df = df.replace('江北新区直管区','江北')
        df.set_index('rank', inplace=True)
        df = df.style.applymap(color_of_price,  subset=['均价']).bar(subset=['面积'])
        return df
    
sc =  ShiChang()

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


#### 走势

In [2]:
sc.trend_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,sale,sold,price
201706,90.65,56.83,25472.0
201707,78.50,67.76,24709.0
201708,35.19,38.25,22895.0
201709,23.72,76.09,27603.0
201710,12.49,41.43,24024.0
201711,126.97,53.42,24717.0
201712,132.69,89.05,26590.0
201801,27.08,77.01,26728.0
201802,13.84,36.70,25196.0
201803,18.54,29.83,24765.0


In [15]:
print(sc.trend_argue)
print(sc.trend_average)

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


上市37.88万㎡，环比增长11%。成交35.62万㎡，环比下降6%。成交均价25861元/㎡，环比下降7%。截止5月底全市库存399万㎡，去化周期7.0个月（按12个月计算）。
近一年月均供应：53万㎡ 近一年月均成交：53万㎡
None


#### 库存

In [23]:
sc.stock_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


,面积,速度_3,速度_12,速度_36
90-㎡,69.52,2.62,0.45,0.08
90-120㎡,94.50,3.55,0.52,0.10
120-144㎡,81.23,3.46,0.46,0.12
144-180㎡,33.49,4.90,0.69,0.18
180+㎡,120.48,6.00,1.53,0.27


In [6]:
sc.stock_argue

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


'截止本月底库存：399万㎡   去化周期—按近3个月计算：3.9个月   按近12个月计算：0.6个月   按近36个月计算：0.1个月'

#### 交叉

In [7]:
sc.cross_table

d:\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1297: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['8']; using no arguments.
  (coltype, args))


#### 排行榜

In [35]:
sc.rank

,popularizename,板块,面积,件数,金额,均价
rank,,,,,,
1,南京溧水万达广场,溧水,6.01,537,7.06,11758
2,融侨悦城,江宁,3.88,369,8.83,22772
3,天岳城,溧水,2.74,247,2.22,8118
4,恒大金碧天下,溧水,2.58,188,2.02,7846
5,万达茂,仙林,2.18,706,3.39,15517
6,世茂荣里,江北,2.13,222,5.14,24107
7,万科城,溧水,2.09,194,2.54,12164
8,御澜府,江北,2.05,194,5.22,25477
9,海赋尚城,城北,2.04,229,5.23,25641


#### 竞品

In [69]:
projects = ['华润国际社区','世茂荣里','中建国熙台','三金鑫宁府','雅居乐滨江国际','观山悦']

def get_one(prj):
    df = ws.query('month_sold')\
            .filter(popname=prj, date_range=[m.date, m.date], usage=SPZZ)\
            .group('popularizename', ['件数','面积','金额']).df()
    df['均价'] = df.金额/df.面积
    
    if len(df):
        print(f"{prj}签约{df.at[0,'件数']}套, 均价{wan(df.at[0,'均价']):.1f}元/㎡。")

    return df
    
df = pd.concat((get_one(x) for x in projects[:-1]), ignore_index=True)
df[['popularizename','件数','均价']].style.format({'均价':'{:.0f}'})

华润国际社区签约63套, 均价3.0元/㎡。
世茂荣里签约222套, 均价2.4元/㎡。
中建国熙台签约1套, 均价3.3元/㎡。
三金鑫宁府签约11套, 均价2.6元/㎡。
雅居乐滨江国际签约40套, 均价2.6元/㎡。


,popularizename,件数,均价
0,华润国际社区,63,29638
1,世茂荣里,222,24107
2,中建国熙台,1,32788
3,三金鑫宁府,11,25961
4,雅居乐滨江国际,40,26511


In [68]:
prj = projects[-1]
df = ws.query('month_sold')\
        .filter(popname=prj, date_range=[m.date, m.date], usage=SPZZ)\
        .group(['popularizename','功能'], ['件数','面积','金额']).df()
df['均价'] = df.金额/df.面积

for idx, row in df.iterrows():
    print(f"{row.功能}{row.件数}套, 均价{wan(row.均价):.1f}元/㎡。")

df[['popularizename','件数','均价']].style.format({'均价':'{:.0f}'})

多层住宅1套, 均价2.8元/㎡。
小高层住宅10套, 均价2.5元/㎡。


,popularizename,件数,均价
0,观山悦,1,28386
1,观山悦,10,25089
